In [2]:
%pylab inline
%load_ext autoreload


Populating the interactive namespace from numpy and matplotlib


In [ ]:
from math import sqrt
from skimage import data
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.color import rgb2gray, gray2rgb
from skimage.io import imread, imshow
import cv2

import pycocotools

from pycocotools import mask as mask_coco

import matplotlib.pyplot as plt


image = imread("TEETH_DATASET/masks/cate1-00001.bmp")
image_gray = rgb2gray(image)
image = gray2rgb(image)

ret, thresh = cv2.threshold(image_gray, 0, 255, 0)
im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

for i in range(len(contours)):
    img_cont = np.zeros(image.shape, dtype=image.dtype)

    cv2.drawContours(img_cont, contours, i, (255,255,255), -1)

    #imshow(img_cont)

    bimask = np.zeros(img_cont.shape[0:2], dtype=np.uint8)

    bimask = img_cont[:,:, 0]

    #print(len(contours))
    
    cnt = contours[2]
    x,y,w,h = cv2.boundingRect(cnt)
    cv2.namedWindow('image', cv2.WINDOW_NORMAL)
    cv2.imshow('image',bimask)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    #break
    #print(x,y,w,h)
#M = cv2.moments(cnt)
#print( cnt )

#mask_coco.encode(img_cont)


In [ ]:





'''# Import libraries
import numpy as np # linear algebra
from PIL import Image

# Fast run length encoding
def rle (img):
    flat_img = img.flatten()
    flat_img = np.where(flat_img > 0.5, 1, 0).astype(np.uint8)

    starts = np.array((flat_img[:-1] == 0) & (flat_img[1:] == 1))
    ends = np.array((flat_img[:-1] == 1) & (flat_img[1:] == 0))
    starts_ix = np.where(starts)[0] + 2
    ends_ix = np.where(ends)[0] + 2
    lengths = ends_ix - starts_ix
    
    return starts_ix, lengths

mask = bimask
mask_rle = rle(mask)
print(mask_rle)
'''

In [18]:
%autoreload 2
import numpy as np
import random
import os
import fnmatch
import datetime
from sklearn import model_selection
from PIL import Image
import pycococreatortool_teeth as cococreator_teeth

# dataset_path = '/workspace/teeth_net/classified_panoramic_X-ray_images_dataset/mask'
# dataset_path_save = '/workspace/teeth_net/classified_panoramic_X-ray_images_dataset'

#accuracy_score
#f1_score
#precision_score
#recall_score
#specificity_score

dataset_path = '/workspace/teeth_net/temp_teeth_daquelaveznaofoimasagoravai/accuracy_score/best/expected'
dataset_path_save = '/workspace/teeth_net/temp_teeth_daquelaveznaofoimasagoravai/accuracy_score/best/'


images_path = os.listdir(dataset_path)
#images_path = fnmatch.filter(images_path, '*.png')
train_data, val_data = model_selection.train_test_split(images_path, train_size=0, shuffle=True)
arr_dict_images_train = []
arr_dict_images_val = []
arr_dict_annotation_train = []
arr_dict_annotation_val = []
annotation_id = 0

path_dir_train = '/workspace/teeth_net/temp_teeth_daquelaveznaofoimasagoravai/accuracy_score/best/expected'#"/workspace/teeth_net/TEETH_DATASET/masks"
paths_files_train = [f.split(".")[0] for f in os.listdir(path_dir_train)]


#images_path = [f for f in images_path if f.split('.')[0] not in paths_files_train]
print(images_path)
for img_id, img_path in enumerate(images_path):
    img_fullpath = os.path.join(dataset_path, img_path)
    binary_mask = np.asarray(Image.open(img_fullpath).convert('1')).astype(np.uint8)
    
    # Create dict images_info
    dict_images = cococreator_teeth.create_image_info(
        image_id = img_id,
        file_name = img_path.replace('.png', '.jpg'), 
        image_size = binary_mask.shape,
        date_captured = datetime.datetime.utcnow().isoformat(' '))
    
    if img_path in train_data: arr_dict_images_train.append(dict_images)
    elif img_path in val_data: arr_dict_images_val.append(dict_images)
    
    # Create dict annotations_info for each image
    annotations = cococreator_teeth.get_teeth_masks_test(img_fullpath)
    for ann in annotations:
        dict_annotation_info = cococreator_teeth.create_annotation_info(
                annotation_id = annotation_id, 
                image_id = img_id, 
                binary_mask = ann,
                class_id = 1,
                is_crowd = 1,
                image_size = binary_mask.shape,
                tolerance = 2, 
                bounding_box = None)
        annotation_id +=1 
        if dict_annotation_info is None: continue
        if img_path in train_data: arr_dict_annotation_train.append(dict_annotation_info)
        elif img_path in val_data: arr_dict_annotation_val.append(dict_annotation_info)
        
print(len(arr_dict_images_train), len(arr_dict_images_val))
# Build JSON for subsets
#dataset_path = os.path.join(dataset_path, os.pardir) # Go Back 1 folder
#print(arr_dict_images_train)
cococreator_teeth.build_coco_format(images=arr_dict_images_train, 
                                    annotations=arr_dict_annotation_train, 
                                    dataset_path=dataset_path_save, 
                                    subset_dir='train2018')
cococreator_teeth.build_coco_format(images=arr_dict_images_val, 
                                    annotations=arr_dict_annotation_val, 
                                    dataset_path=dataset_path_save, 
                                    subset_dir='test2018')


['cate5-00091.png']
0 1
JSON created...OK in:  %s /workspace/teeth_net/temp_teeth_daquelaveznaofoimasagoravai/accuracy_score/best/annotations/instances_train2018.json
JSON created...OK in:  %s /workspace/teeth_net/temp_teeth_daquelaveznaofoimasagoravai/accuracy_score/best/annotations/instances_test2018.json


/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


{'annotations': [{'area': 18343,
   'bbox': [397.0, 563.0, 501.0, 168.0],
   'category_id': 1,
   'height': 1024,
   'id': 0,
   'image_id': 0,
   'iscrowd': 1,
   'segmentation': {'counts': [407203,
     19,
     999,
     29,
     989,
     36,
     982,
     42,
     976,
     48,
     969,
     55,
     958,
     65,
     953,
     71,
     952,
     71,
     949,
     73,
     948,
     74,
     950,
     71,
     944,
     77,
     946,
     74,
     949,
     72,
     952,
     70,
     953,
     67,
     957,
     62,
     962,
     57,
     967,
     51,
     973,
     44,
     980,
     39,
     986,
     35,
     989,
     8,
     3,
     21,
     1005,
     2,
     6,
     11,
     1,
     3,
     16,
     50,
     946,
     81,
     943,
     82,
     941,
     83,
     941,
     83,
     941,
     83,
     941,
     83,
     940,
     84,
     940,
     84,
     940,
     84,
     939,
     85,
     939,
     86,
     938,
     86,
     938,
     86,
     938,
     86,
  

In [9]:
arr_dict_annotation_train

[]